In [1]:
import cv2
import numpy as np
from object_detection import ObjectDetection
import math

In [ ]:
# Initialize Object Detection
od = ObjectDetection()

cap = cv2.VideoCapture("los_angeles.mp4")

# Initialize count
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

while True:
    ret, frame = cap.read()
    count += 1
    if not ret:
        break

    # Point current frame
    center_points_cur_frame = []

    # Detect objects on frame
    (class_ids, scores, boxes) = od.detect(frame)
    for box in boxes:
        (x, y, w, h) = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        center_points_cur_frame.append((cx, cy))
        #print("FRAME N°", count, " ", x, y, w, h)

        # cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Only at the beginning we compare previous and current frame
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:

        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            # Remove IDs lost
            if not object_exists:
                tracking_objects.pop(object_id)

        # Add new IDs found
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1

    for object_id, pt in tracking_objects.items():
        cv2.circle(frame, pt, 5, (0, 0, 255), -1)
        cv2.putText(frame, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 2)

    print("Tracking objects")
    print(tracking_objects)


    print("CUR FRAME LEFT PTS")
    print(center_points_cur_frame)


    cv2.imshow("Frame", frame)

    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


Loading Object Detection
Running opencv dnn with YOLOv4
Tracking objects
{}
CUR FRAME LEFT PTS
[(571, 891), (437, 742), (1750, 635), (761, 648), (1018, 647), (928, 533), (856, 565), (612, 473), (881, 473), (1336, 980), (753, 463), (1118, 436), (942, 458), (1877, 605), (642, 435), (688, 450), (1268, 433), (1424, 466)]
Tracking objects
{0: (567, 905), 1: (434, 750), 2: (930, 534), 3: (857, 567), 4: (1019, 650), 5: (761, 655), 6: (612, 474), 7: (881, 473), 8: (753, 463), 9: (1867, 590), 10: (641, 436), 11: (687, 451), 12: (943, 459), 13: (1347, 984), 14: (1114, 435), 15: (1417, 465)}
CUR FRAME LEFT PTS
[(567, 905), (434, 750), (1724, 626), (930, 534), (857, 567), (1019, 650), (761, 655), (612, 474), (881, 473), (753, 463), (1867, 590), (641, 436), (687, 451), (943, 459), (1347, 984), (1114, 435), (1417, 465)]
Tracking objects
{0: (564, 920), 1: (427, 756), 2: (930, 536), 3: (858, 571), 4: (1021, 651), 5: (762, 659), 6: (612, 475), 7: (882, 475), 8: (753, 465), 9: (1857, 585), 10: (641, 43